# **Hashed Features Design Patter**

### ***Loading Libraries***

In [ ]:
# Operating System
import os

# Data Manipualtion
import pandas as pd

# SciPy
import scipy
from scipy import stats

# TensorFlow
import tensorflow as tf

# Scikit-Learn
import sklearn
from sklearn.utils import shuffle
from sklearn import datasets, linear_model

# Google Libraries
from google.colab import auth
from google.cloud import bigquery

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
auth.authenticate_user()

project_id = 'core-catalyst-425922-v9'
os.environ['GOOGLE_CLOUD_PROJECT'] = project_id

client = bigquery.Client(project=project_id)

In [ ]:
%%bigquery df --project core-catalyst-425922-v9
CREATE TEMPORARY FUNCTION hashed(airport STRING, numbuckets INT64) AS (
   ABS(MOD(FARM_FINGERPRINT(airport), numbuckets))
);

WITH airports AS (
SELECT
   DISTINCT(departure_airport)
FROM `bigquery-samples.airline_ontime_data.flights`
)

SELECT
   departure_airport,
   hashed(departure_airport, 3) AS hash3,
   hashed(departure_airport, 10) AS hash10,
   hashed(departure_airport, 1000) AS hash1000
FROM airports


Query is running:   0%|          |

Downloading:   0%|          |

In [ ]:
df.head(n=10)

,departure_airport,hash3,hash10,hash1000
0,CVG,1,6,846
1,MCO,2,3,193
2,DFW,2,8,958
3,BWI,2,9,259
4,TPA,2,4,74
5,ONT,0,1,561
6,DLG,0,7,537
7,OME,2,3,213
8,MLI,1,6,476
9,BGR,0,4,304


In [ ]:
len(df)

347

In [ ]:
%%bigquery df --project core-catalyst-425922-v9
SELECT
   departure_airport, COUNT(1) AS num_flights
FROM `bigquery-samples.airline_ontime_data.flights`
GROUP BY departure_airport
ORDER BY num_flights ASC
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

### **Likelihood of Collision**

In [ ]:
def calc_collision_prob(num_total, num_hash):
    no_collision_prob = 1.0
    for i in range(num_total):
        collision_likelihood = float(i) / num_hash
        no_collision_prob *= (1 - collision_likelihood)
    return 1 - no_collision_prob


data = []

for num_hash in [3, 10, 100, 1000, 10000, 100000]:
    data.append([num_hash,
                 len(df)/num_hash,
                 calc_collision_prob(len(df), num_hash)
                ])
prob = pd.DataFrame(data, columns=['num_hash_buckets', 'entries_per_bucket', 'collision_prob'])

prob

,num_hash_buckets,entries_per_bucket,collision_prob
0,3,3.333333,1.000000
1,10,1.000000,0.999637
2,100,0.100000,0.371843
3,1000,0.010000,0.044139
4,10000,0.001000,0.004491
5,100000,0.000100,0.000450


In [ ]:
calc_collision_prob(5, 1000)

0.009965049976000118

### **Airport's Shared Buckets**

In [ ]:
%%bigquery df --project core-catalyst-425922-v9
CREATE TEMPORARY FUNCTION hashed(airport STRING, numbuckets INT64) AS (
   ABS(MOD(FARM_FINGERPRINT(airport), numbuckets))
);

WITH airports AS (
SELECT
   departure_airport, COUNT(1) AS num_flights
FROM `bigquery-samples.airline_ontime_data.flights`
GROUP BY departure_airport
)

SELECT
   departure_airport, num_flights
FROM airports
WHERE hashed(departure_airport, 100) = hashed('ORD', 100)

Query is running:   0%|          |

Downloading:   0%|          |